# Obtain Historical Data

In [4]:
from quantrocket.history import create_usstock_db

create_usstock_db("usstock-free-1d", bar_size="1 day", free=True)

HTTPError: ('400 Client Error: BAD REQUEST for url: http://houston/history/databases/usstock-free-1d?vendor=usstock&bar_size=1+day&free=True', {'status': 'error', 'msg': 'quantrocket.v2.history.usstock-free-1d.sqlite already exists'})

In [5]:
from quantrocket.history import collect_history

collect_history("usstock-free-1d")

{'status': 'the historical data will be collected asynchronously'}

In [7]:
from quantrocket.master import get_securities

# by specifying sec_types='STK' we exclude the ETFs we obtained
securities = get_securities(vendors="usstock", sec_types="STK")
securities.head()

,Symbol,Exchange,Country,Currency,SecType,Etf,Timezone,Name,PriceMagnifier,Multiplier,Delisted,DateDelisted,LastTradeDate,RolloverDate
Sid,,,,,,,,,,,,,,
FIBBG000B9XRY4,AAPL,XNAS,US,USD,STK,False,America/New_York,APPLE INC,1,1,False,NaT,NaT,NaT
FIBBG000BFWKC0,MON,XNYS,US,USD,STK,False,America/New_York,MONSANTO CO,1,1,True,2018-06-06,NaT,NaT
FIBBG000BKZB36,HD,XNYS,US,USD,STK,False,America/New_York,HOME DEPOT INC,1,1,False,NaT,NaT,NaT
FIBBG000BMHYD1,JNJ,XNYS,US,USD,STK,False,America/New_York,JOHNSON & JOHNSON,1,1,False,NaT,NaT,NaT
FIBBG000BPH459,MSFT,XNAS,US,USD,STK,False,America/New_York,MICROSOFT CORP,1,1,False,NaT,NaT,NaT


# Create Universe

In [8]:
from quantrocket.master import create_universe

create_universe("usstock-free", sids=securities.index.tolist())

{'code': 'usstock-free', 'provided': 8, 'inserted': 8, 'total_after_insert': 8}

In [9]:
listed_securities = securities[securities.Delisted==False]

listed_securities = listed_securities[["Symbol", "Exchange", "Name"]]
listed_securities.head()

,Symbol,Exchange,Name
Sid,,,
FIBBG000B9XRY4,AAPL,XNAS,APPLE INC
FIBBG000BKZB36,HD,XNYS,HOME DEPOT INC
FIBBG000BMHYD1,JNJ,XNYS,JOHNSON & JOHNSON
FIBBG000BPH459,MSFT,XNAS,MICROSOFT CORP
FIBBG000GZQ728,XOM,XNYS,EXXON MOBIL CORP


In [10]:
create_universe("usstock-free-active", sids=listed_securities.index.tolist())

{'code': 'usstock-free-active',
 'provided': 6,
 'inserted': 6,
 'total_after_insert': 6}

# Collect real-time data

In [11]:
from quantrocket.realtime import create_ibkr_tick_db

create_ibkr_tick_db("fang-stk-tick", universes="fang-stk",
                        fields=[
                            "LastPrice", "Volume", "BidPrice",
                            "AskPrice", "BidSize", "AskSize"
                        ]
                   )

{'status': 'successfully created tick database fang-stk-tick'}

In [14]:
from quantrocket.realtime import list_databases
list_databases()

{'fang-stk-tick': []}

In [18]:
from quantrocket.realtime import collect_market_data

collect_market_data("fang-stk-tick", sids=["FIBBG000B9XRY4", "FIBBG000BDTBL9"])

{'status': 'the market data will be collected until canceled'}

In [19]:
from quantrocket.realtime import get_active_collections

get_active_collections()

{'ibkr': {'fang-stk-tick': 2}}

In [20]:
!quantrocket flightlog stream -d

quantrocket_account_1|can't yet collect ECB exchange rates because no base currencies found in quantrocket.v2.account.balance.sqlite
quantrocket_houston_1|172.19.0.2 - - [02/Sep/2022:18:07:53 +0000] "GET /ibg1/gateway HTTP/1.1" 200 22 "-" "python-requests/2.25.1"
quantrocket_houston_1|172.19.0.22 - - [02/Sep/2022:18:07:53 +0000] "GET /ibgrouter/gateways?status=running HTTP/1.1" 200 3 "-" "python-urllib3/1.26.6"
quantrocket_license-service_1|fetching license profile for 727c........5a67
        quantrocket_houston_1|172.19.0.2 - - [02/Sep/2022:18:08:21 +0000] "GET /ibg1/gateway HTTP/1.1" 200 22 "-" "python-requests/2.25.1"
        quantrocket_houston_1|172.19.0.22 - - [02/Sep/2022:18:08:21 +0000] "GET /ibgrouter/gateways?status=running HTTP/1.1" 200 3 "-" "python-urllib3/1.26.6"
        quantrocket_houston_1|172.19.0.2 - - [02/Sep/2022:18:08:53 +0000] "GET /ibg1/gateway HTTP/1.1" 200 22 "-" "python-requests/2.25.1"
        quantrocket_houston_1|172.19.0.22 - - [02/Sep/2022:18:08:53 +000

In [22]:
from quantrocket.realtime import cancel_market_data

cancel_market_data(cancel_all=True)

{}